In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data

In [81]:
dataset = pd.read_csv('https://raw.githubusercontent.com/revanthmadasu/LifestyleWellbeing/master/Wellbeing_and_lifestyle_data_Kaggle.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

Converting categorial data into numerical

Understanding which objects to convert

In [82]:
object_dataset = dataset.select_dtypes(include=['object']).copy()
object_dataset.head()

,Timestamp,DAILY_STRESS,AGE,GENDER
0,7/7/15,2,36 to 50,Female
1,7/7/15,3,36 to 50,Female
2,7/7/15,3,36 to 50,Female
3,7/7/15,3,51 or more,Female
4,7/7/15,1,51 or more,Female


Converting Timestamp using label encoding.
"Finding and replacing" method would take forever.
"One-hot encoder" method would creat a lot more unnecessary columns


In [83]:
dataset['Timestamp'] = dataset['Timestamp'].astype('category')
dataset['Timestamp'] = dataset['Timestamp'].cat.codes
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5607,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5607,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5607,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5607,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5607,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


Using "Finding and replacing" method, since there are values only from 0 to 5, with a arbitrary value of 1/1/2000 in row 10007

In [84]:
dataset['DAILY_STRESS'] = dataset['DAILY_STRESS'].replace(['0','1','2','3','4','5'], [0,1,2,3,4,5])
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5607,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5607,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5607,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5607,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5607,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


Using get_dummies method for 'AGE' & 'GENDER' category

In [85]:
dummy_col_df = pd.get_dummies(dataset, columns = ['AGE', 'GENDER'])
dummy_col_df.columns

Index(['Timestamp', 'FRUITS_VEGGIES', 'DAILY_STRESS', 'PLACES_VISITED',
       'CORE_CIRCLE', 'SUPPORTING_OTHERS', 'SOCIAL_NETWORK', 'ACHIEVEMENT',
       'DONATION', 'BMI_RANGE', 'TODO_COMPLETED', 'FLOW', 'DAILY_STEPS',
       'LIVE_VISION', 'SLEEP_HOURS', 'LOST_VACATION', 'DAILY_SHOUTING',
       'SUFFICIENT_INCOME', 'PERSONAL_AWARDS', 'TIME_FOR_PASSION',
       'WEEKLY_MEDITATION', 'WORK_LIFE_BALANCE_SCORE', 'AGE_21 to 35',
       'AGE_36 to 50', 'AGE_51 or more', 'AGE_Less than 20', 'GENDER_Female',
       'GENDER_Male'],
      dtype='object')

In [87]:
dummy_col_df[['Timestamp', 'FRUITS_VEGGIES', 'DAILY_STRESS', 'PLACES_VISITED',
       'CORE_CIRCLE', 'SUPPORTING_OTHERS', 'SOCIAL_NETWORK', 'ACHIEVEMENT',
       'DONATION', 'BMI_RANGE', 'TODO_COMPLETED', 'FLOW', 'DAILY_STEPS',
       'LIVE_VISION', 'SLEEP_HOURS', 'LOST_VACATION', 'DAILY_SHOUTING',
       'SUFFICIENT_INCOME', 'PERSONAL_AWARDS', 'TIME_FOR_PASSION',
       'WEEKLY_MEDITATION', 'AGE_21 to 35',
       'AGE_36 to 50', 'AGE_51 or more', 'AGE_Less than 20', 'GENDER_Female',
       'GENDER_Male', 'WORK_LIFE_BALANCE_SCORE']]

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE_21 to 35,AGE_36 to 50,AGE_51 or more,AGE_Less than 20,GENDER_Female,GENDER_Male,WORK_LIFE_BALANCE_SCORE
0,5607,3,2,2,5,0,5,2,0,1,...,4,0,5,0,1,0,0,1,0,609.5
1,5607,2,3,4,3,8,10,5,2,2,...,3,2,6,0,1,0,0,1,0,655.6
2,5607,2,3,3,4,4,10,3,2,2,...,4,8,3,0,1,0,0,1,0,631.6
3,5607,3,3,10,3,10,7,2,5,2,...,5,2,0,0,0,1,0,1,0,622.7
4,5607,5,1,3,3,10,4,2,4,2,...,8,1,5,0,0,1,0,1,0,663.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15967,3052,3,3,0,4,0,10,0,4,2,...,5,2,5,0,0,1,0,1,0,644.5
15968,3053,3,3,6,8,7,4,6,3,1,...,10,5,8,1,0,0,0,1,0,714.9
15969,3054,4,3,0,10,10,8,6,5,1,...,10,1,10,1,0,0,0,0,1,716.6
15970,3055,1,1,10,8,2,7,3,2,1,...,1,6,8,1,0,0,0,1,0,682.0


In [ ]:
print(X[0,:])

['7/7/15' 3 '2' 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 '36 to 50' 'Female']


In [ ]:
print(X)

[['7/7/15' 3 '2' ... 5 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 6 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 3 '36 to 50' 'Female']
 ...
 ['3/14/21 8:35' 4 '3' ... 10 '21 to 35' 'Male']
 ['3/14/21 8:43' 1 '1' ... 8 '21 to 35' 'Female']
 ['3/14/21 9:03' 5 '4' ... 4 '21 to 35' 'Female']]


In [ ]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


Handling missing data

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(X[:, 1:3])
# X[:, 1:3] = imputer.transform(X[:, 1:3])